### Dimensionality Reduction

first we should try to train system on original data before reduction

reducing noise and filtering unnecessary details will result in higher performance (speed up training)

Reducing the dimensionality could help in visuals such as detecting patterns (clusters)

**Projection:** 3D space -> 2D space

**Manifold:** Swiss Roll data example from 3D to unrolling it onto 2D space

### PCA Principal Component Analysis

the most popular dimensionality reduction algorithm

identifies the axis that accounts for the largest amount of variance in the trainging set

finds second axis orthogonal to the first one that accounts for the largest amount of the remaining variance (as many axes as the number of dimensions)

To find the principal components of a training set?

using standard matrix factorization technique called sinular value decomposition (SVD)
decompose X onto UEV^T

In [6]:
import numpy as np

# Create a small 3D dataset (5 points in 3D space)
X = np.array([
    [2, 3, 5],
    [3, 5, 7],
    [5, 8, 11],
    [7, 10, 13],
    [9, 12, 15]
])

# Center the data
X_centered = X - X.mean(axis=0)

print(X.mean(axis=0))
# Perform SVD
U, s, Vt = np.linalg.svd(X_centered)

# Principal components
c1 = Vt[0]  # First principal component
c2 = Vt[1]  # Second principal component

print("Principal Component 1:", c1)
print("Principal Component 2:", c2)

[ 5.2  7.6 10.2]
Principal Component 1: [0.45795989 0.58739374 0.66726407]
Principal Component 2: [-0.85099447  0.0726221   0.52012926]


Projects training set onto a plane:

In [7]:
W2 = Vt[:2].T
X2D = X_centered @ W2

reduce dimensionality of dataset down to two dimensions
(automatically takes care of centering the data)

In [8]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

Ratio of the proportion of the dataset's variance

In [9]:
pca.explained_variance_ratio_

array([0.99541491, 0.00428147])

0.9954149 variance lies on first PC, 0.00428 second PC

Instead of arbitrarily choosing the number of dimensions to reduce down to, it is simpler to choose the number of dimensions that add up to a sufficiently large portion of the variance ~95%

In [10]:
from sklearn.datasets import load_digits

digits = load_digits()
X, y = digits.data, digits.target

X_train, X_test, y_train, y_test = X[:1500], X[1500:], y[:1500], y[1500:]

pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

In [11]:
# variance to perserve:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

In [12]:
pca.n_components_

28

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline

clf = make_pipeline(PCA(random_state=42), RandomForestClassifier(random_state=42))

param_distrib = {
    "pca__n_components": np.arange(10, 80),
    "randomforestclassifier__n_estimators": np.arange(50, 500)
}

rnd_search = RandomizedSearchCV(clf, param_distrib, n_iter=10, cv=3, random_state=42)
rnd_search.fit(X_train[:1000], y_train[:1000])

/Users/mathias/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
3 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/mathias/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mathias/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/mathias/Library/Python/3.9/lib/python/site-packages/sklearn/pipeline.py", line 471, in fit
    Xt = self._fi

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(random_state=42))]),
                   param_distributions={'pca__n_components': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       6...
       414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426,
       427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439,
       440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465,
       466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478,
       479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491,
       492, 493, 494, 495, 496, 497, 498, 499])},
                   random_state=42)

In [19]:
rnd_search.best_params_

{'randomforestclassifier__n_estimators': 304, 'pca__n_components': 62}

This ^ reduced number of dimensions to 62

reconstructed data (compressed then decompressed) is called reconstruction error

In [20]:
X_recovered = pca.inverse_transform(X_reduced)

$$
\mathbf{X}_{\text{reconstructed}} = \mathbf{Z} \mathbf{W}^T + \mathbf{\mu}
$$

If mean-centering was not applied before PCA, the equation simplifies to:

$$
\mathbf{X}_{\text{reconstructed}} = \mathbf{Z} \mathbf{W}^T
$$

In [24]:
# randomized SVD is dramatically faster than full SVD
# (uses randomized by default if components 500+)
rnd_pca = PCA(n_components=32, svd_solver="randomized", random_state=42)
X_reduced = rnd_pca.fit_transform(X_train)

In [27]:
# mini  batch
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=5)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

In [28]:
# with numpy's memmap class you can manipulate a large array stored in a binary file
# X_train would typically not fit in memory so we would load it chunk by chunk:
filename = "my_mnist.mmap"
X_mmap = np.memmap(filename, dtype="float32", mode="write", shape=X_train.shape)
X_mmap[:] = X_train
X_mmap.flush()

In [30]:
# using this method we can call .fit() instead of .partial_fit()
X_mmap = np.memmap(filename, dtype="float32", mode="readonly").reshape(-1, 32)
batch_size = X_mmap.shape[0]
inc_pca = IncrementalPCA(n_components=32, batch_size=batch_size)
inc_pca.fit(X_mmap)

IncrementalPCA(batch_size=3000, n_components=32)

PCA can be really slow for large datasets we can consider using random projection instead

### Random Projection

Think of a shadow it's a lower-dimensional representation of a 3D object, but the overall shape is still recognizable. The lemma (William Johnson and Joram Lindenstrauss) ensures that the "shadow" doesn’t distort the relative distances too much.

reduced dimensions while preserving distances

## **Example Calculation**
Suppose we have:
- \( m = 1000 \) points
- \( \epsilon = 0.1 \) (10% allowable distortion)

Using the formula:

$$
d \geq \frac{4 \log 1000}{\frac{1}{2} (0.1)^2 - \frac{1}{3} (0.1)^3}
$$

Approximating values:

$$
\log 1000 \approx 6.91, \quad \frac{1}{2} (0.1)^2 = 0.005, \quad \frac{1}{3} (0.1)^3 = 0.00033
$$

$$
d \geq \frac{4 \times 6.91}{0.005 - 0.00033} = \frac{27.64}{0.00467} \approx 5917
$$

Thus, we need about **5917 dimensions** to ensure distances are preserved within **10% distortion** for 1000 points.

In [31]:
from sklearn.random_projection import johnson_lindenstrauss_min_dim

m, e = 5_000, 0.1
d = johnson_lindenstrauss_min_dim(m, eps=e)
d

7300

In [32]:
# random matrix P of shape [d, n], project n dimensions down to d:
n = 20_000
np.random.seed(42)
P = np.random.randn(d, n) / np.sqrt(d) # Standard dev = square root of variance

X = np.random.randn(m, n)
X_reduced = X @ P.T

`GaussianRandomProjection` does this and uses `johnson_lindenstrauss_min_dim` when calling `fit()`

random projection matrix 𝑃 is generated independently of the actual dataset 𝑋

In [33]:
from sklearn.random_projection import GaussianRandomProjection

gaussian_rnd_proj = GaussianRandomProjection(eps=e, random_state=42)
X_reduced = gaussian_rnd_proj.fit_transform(X)

Its perferable to use `SparseRandomProjection` faster, less memory

The ratio r of nonzero items in the sparse random matrix is called its density

inverse transform, compute the pseudo-inverse of the components matrix using `pinv()`, then multiply the reduced data by the transpose of the pseudo-inverse:

In [35]:
components_pinv = np.linalg.pinv(gaussian_rnd_proj.components_)
X_recovered = X_reduced @ components_pinv.T

intresting to look up Locality sensitive hashing with fruit flys and their olfactory inputs -> output of neurons

### Locally Linear Embedding LLE

good for unrolling twisted manifolds, (swiss roll dataset) distances are locally well preserved, but on a larger scale are not

In [36]:
from sklearn.datasets import make_swiss_roll
from sklearn.manifold import LocallyLinearEmbedding

X_swiss, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_unrolled = lle.fit_transform(X_swiss)

Here's a step-by-step explanation of the Locally Linear Embedding (LLE) algorithm with a simple example in Markdown format:

markdown
Copy
Edit
# Locally Linear Embedding (LLE) Algorithm: Step-by-Step

## Step 1: Input Data
Consider a simple 3D dataset with 5 points:

| Point | X  | Y  | Z  |
|-------|----|----|----|
| P1    | 1  | 2  | 3  |
| P2    | 2  | 3  | 4  |
| P3    | 3  | 5  | 6  |
| P4    | 5  | 7  | 8  |
| P5    | 8  | 9  | 10 |

Our goal is to reduce this dataset from **3D to 2D** while preserving local geometry.

---

## Step 2: Find Nearest Neighbors
Choose **k = 2** (each point is reconstructed using its 2 nearest neighbors).

Nearest neighbors based on Euclidean distance:

- **P1** → P2, P3
- **P2** → P1, P3
- **P3** → P2, P4
- **P4** → P3, P5
- **P5** → P4, P3

---

## Step 3: Compute Reconstruction Weights
Each point is reconstructed as a linear combination of its neighbors.

For each point \( x_i \), we solve for weights \( W_{ij} \) minimizing:

$$
\sum_i || x_i - \sum_{j \in N(i)} W_{ij} x_j ||^2
$$

Example:  
For **P1** (neighbors: P2, P3), solve:

$$
P1 = W_{12} P2 + W_{13} P3
$$

Subject to the constraint:

$$
W_{12} + W_{13} = 1
$$

Solving for weights (assuming Euclidean distances):

- \( W_{12} = 0.7 \), \( W_{13} = 0.3 \)

Repeat for all points.

---

## Step 4: Compute Low-Dimensional Embedding
Find lower-dimensional representations \( Y_i \) by solving:

$$
\sum_i || Y_i - \sum_{j \in N(i)} W_{ij} Y_j ||^2
$$

Subject to centering constraints:

$$
\sum_i Y_i = 0
$$

Solving this eigenvalue problem, we get **2D coordinates** for each point.

---

## Step 5: Output the 2D Representation

Example output (2D representation):

| Point | Y1  | Y2  |
|-------|----|----|
| P1    | 1.1  | -0.5 |
| P2    | 2.0  | -0.2 |
| P3    | 3.2  | 0.1  |
| P4    | 4.5  | 0.3  |
| P5    | 5.7  | 0.8  |

These 2D points preserve the local relationships of the original 3D data.

---

## Summary of LLE Algorithm
1. **Find Nearest Neighbors** → Determine k-nearest neighbors for each point.
2. **Compute Reconstruction Weights** → Solve for weights that best reconstruct each point using its neighbors.
3. **Compute Low-Dimensional Embedding** → Solve an eigenvalue problem to obtain the lower-dimensional representation.

LLE is useful for **nonlinear dimensionality reduction** while preserving local geometry.

### other dimentionality reduction techniques:

- MDS
- Isomap
- TSNE
- Linear discriminant analysis (LDA)